In [1]:
import numpy as np
import pandas as pd
import re
import random
import math

# Importing QISKit
from qiskit import QuantumCircuit, QuantumProgram
import Qconfig

# Import basic plotting tools
from qiskit.tools.visualization import plot_histogram

# Quantum program setup
Q_program = QuantumProgram()
Q_program.set_api(Qconfig.APItoken, Qconfig.config['url']) # set the APIToken and API url

In [2]:
## Input parameters

measurementChoicesLength = 500 # length of the strings that cointain measurement choices
evePresence = True # Presence of an eavesdropper in the channel
evePresencePercentage = 1.0 # Percentage of Eve's interference

In [3]:
## Creating registers
qr = Q_program.create_quantum_register("qr", 2)
cr = Q_program.create_classical_register("cr", 4)

In [4]:
## Creating a shared entangled state

entangledState = Q_program.create_circuit('entangledState', [qr], [cr])

# bell_11 (singlet) stete for the ibmqx2 backend
entangledState.x(qr[0])
entangledState.x(qr[1])
entangledState.h(qr[0])
entangledState.cx(qr[0],qr[1])

In [5]:
### Creating measurement circuits

## Alice's measurement circuits

# measurement of spin projection onto (1; 0; 0) vector; standard X-measurement
measureA1 = Q_program.create_circuit('measureA1', [qr], [cr])
measureA1.h(qr[0])
measureA1.measure(qr[0],cr[0])

# measurement of spin projection onto (1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto (Z+X)/sqrt(2) eigenstates
measureA2 = Q_program.create_circuit('measureA2', [qr], [cr])
measureA2.s(qr[0])
measureA2.h(qr[0])
measureA2.t(qr[0])
measureA2.h(qr[0])
measureA2.measure(qr[0],cr[0])

# measurement of spin projection onto (0; 0; 1) vector; standard Z-measurement
measureA3 = Q_program.create_circuit('measureA3', [qr], [cr])
measureA3.measure(qr[0],cr[0])

## Bob's measurement circuits

# measurement of spin projection onto (1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto (Z+X)/sqrt(2) eigenstates
measureB1 = Q_program.create_circuit('measureB1', [qr], [cr])
measureB1.s(qr[1])
measureB1.h(qr[1])
measureB1.t(qr[1])
measureB1.h(qr[1])
measureB1.measure(qr[1],cr[1])

# measurement of spin projection onto (0; 0; 1) vector; standard Z-measurement
measureB2 = Q_program.create_circuit('measureB2', [qr], [cr])
measureB2.measure(qr[1],cr[1])

# measurement of spin projection onto (-1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto (Z-X)/sqrt(2) eigenstates
measureB3 = Q_program.create_circuit('measureB3', [qr], [cr])
measureB3.s(qr[1])
measureB3.h(qr[1])
measureB3.tdg(qr[1])
measureB3.h(qr[1])
measureB3.measure(qr[1],cr[1])

## Eve's measuremets

# 1-st qbit identity measurement
ident0 = Q_program.create_circuit('ident0', [qr], [cr])
ident0.iden(qr[0])

# 2-nd qbit identity measurement
ident1 = Q_program.create_circuit('ident1', [qr], [cr])
ident1.iden(qr[1])

# qbit measurement of spin projection onto (1; 0; 0) vector; standard X-measurement
measureEA1 = Q_program.create_circuit('measureEA1', [qr], [cr])
measureEA1.h(qr[0])
measureEA1.measure(qr[0],cr[2])

# qbit measurement of spin projection onto (1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto W = (Z+X)/sqrt(2) eigenstates
measureEA2 = Q_program.create_circuit('measureEA2', [qr], [cr])
measureEA2.s(qr[0])
measureEA2.h(qr[0])
measureEA2.t(qr[0])
measureEA2.h(qr[0])
measureEA2.measure(qr[0],cr[2])

# qbit measurement of spin projection onto (0; 0; 1) vector; standard Z-measurement
measureEA3 = Q_program.create_circuit('measureEA3', [qr], [cr])
measureEA3.measure(qr[0],cr[2])

# measurement of spin projection onto (1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto W = (Z+X)/sqrt(2) eigenstates
measureEB1 = Q_program.create_circuit('measureEB1', [qr], [cr])
measureEB1.s(qr[1])
measureEB1.h(qr[1])
measureEB1.t(qr[1])
measureEB1.h(qr[1])
measureEB1.measure(qr[1],cr[3])

# measurement of spin projection onto (0; 0; 1) vector; standard Z-measurement
measureEB2 = Q_program.create_circuit('measureEB2', [qr], [cr])
measureEB2.measure(qr[1],cr[3])

# measurement of spin projection onto (-1/sqt(2); 0; 1/(sqrt(2)) vector
# projection onto V = (Z-X)/sqrt(2) eigenstates
measureEB3 = Q_program.create_circuit('measureEB3', [qr], [cr])
measureEB3.s(qr[1])
measureEB3.h(qr[1])
measureEB3.tdg(qr[1])
measureEB3.h(qr[1])
measureEB3.measure(qr[1],cr[3])

## Lists of Alice's, Bob's and Eve's measurement circuits

aliceMeasurements = [measureA1, measureA2, measureA3]
bobMeasurements = [measureB1, measureB2, measureB3]
eveMeasurements = [ident0, ident1, measureEA1, measureEA2, measureEA3, measureEB1, measureEB2, measureEB3]

In [6]:
## Alice's and Bob's measurement choice strings

aliceMeasurementChoices = []
bobMeasurementChoices = []

# random strings generation
for i in range(measurementChoicesLength):
    aliceMeasurementChoices.append(random.randint(1, 3))
    bobMeasurementChoices.append(random.randint(1, 3))

In [7]:
len(bobMeasurementChoices)

500

In [8]:
## Eve's optimal measurement choices array

# it is efficient for Eve to measure the spin projections of Alice's and Bob's qubits onto the same direction (A2B1 and A3B2) 

# list of Eve's measurement choices
# the first and the second element of each row represent the measurement of Alice's and Bob's qubit respectively 
# default values of the list are 1 and 2 (applying identity gate)
eveMeasurementChoices = [[1,2] for j in range(measurementChoicesLength)]

eveMersurementPerformed = [0] * measurementChoicesLength

if evePresence == True:
    for j in range(measurementChoicesLength):
        if random.uniform(0, 1) <= evePresencePercentage:
            
            eveMersurementPerformed[j] = 1
            
            if random.uniform(0, 1) <= 0.5: # in 50% of cases perform the A2B1 measurement
                eveMeasurementChoices[j] = [4, 6]
                
            else: # in 50% of cases perform the A3B2 measurement
                eveMeasurementChoices[j] = [5, 7]

In [9]:
circuits = [] # the list in which the created circuits will be stored

for k in range(measurementChoicesLength):
    # create the name of the k-th circuit depending on Alice's, Bob's and Eve's choices of measurement
    circuitName = str(k) + '-A' + str(aliceMeasurementChoices[k]) + '_B' + str(bobMeasurementChoices[k]) + 'E' + str(eveMeasurementChoices[k][0]) + str(eveMeasurementChoices[k][1])
    
    # create the joint measurement circuit
    # add Alice's and Bob's measurement circuits to the singlet state curcuit
    Q_program.add_circuit(circuitName,
                          entangledState + # singlet state circuit
                          eveMeasurements[eveMeasurementChoices[k][0]-1] + # Eve's measurement circuit for Alice's qubit
                          eveMeasurements[eveMeasurementChoices[k][1]-1] + # Eve's measurement circuit for Bob's qubit
                          aliceMeasurements[aliceMeasurementChoices[k]-1] + # measurement circuit of Alice
                          bobMeasurements[bobMeasurementChoices[k]-1] # measurement circuit of Bob
                         )
    
    # add created circuit to the circuits list
    circuits.append(circuitName)

In [10]:
## Backend status check
print(Q_program.get_backend_status('ibmqx2'))

{'available': False, 'busy': False, 'pending_jobs': 27, 'backend': 'ibmqx2'}


In [11]:
## Execute the circuits

# simulator
result = Q_program.execute(circuits, backend='local_qasm_simulator', shots=1)
print(result)

# ibmqx2
#result = Q_program.execute(circuits, backend='ibmqx2', shots=1, timeout=240)
#print(result)

COMPLETED


In [12]:
## Recording the measurement results

aliceResults = [] # Alice's results
bobResults = [] # Bob's results
eveResults = [[0, 0] for j in range(measurementChoicesLength)] # list of zeros by default; if the element is 0, then Eve did not perform a measurement

# Alice's and Bob's search patterns
abPatterns = [
    re.compile('^.*00$'), # search for the '..00' output (Alice obtained -1 and Bob obtained -1)
    re.compile('^.*01$'), # search for the '..01' output
    re.compile('^.*10$'), # search for the '..10' output (Alice obtained -1 and Bob obtained 1)
    re.compile('^.*11$')  # search for the '..11' output
]

# Eve's search patterns
ePatterns = [
    re.compile(r'^00*.$'), # search for the '00..' result
    re.compile(r'^01*.$'),
    re.compile(r'^10*.$'),
    re.compile(r'^11*.$')
]

# Recording results
for k in range(measurementChoicesLength):
    
    res = list(result.get_counts(circuits[k]).keys())[0] # extract a key from the dict and transform it to str; execution result of the k-th circuit
    
    # Alice and Bob
    if abPatterns[0].search(res): # check if the key is '..00' (if the measurement results are -1,-1)
        aliceResults.append(-1) # Alice got the result -1 
        bobResults.append(-1) # Bob got the result -1
    if abPatterns[1].search(res):
        aliceResults.append(1)
        bobResults.append(-1)
    if abPatterns[2].search(res):
        aliceResults.append(-1) 
        bobResults.append(1)
    if abPatterns[3].search(res): # check if the key is '..11' (if the measurement results are 1,1)
        aliceResults.append(1)
        bobResults.append(1)
        
    # Eve
    if eveMersurementPerformed[k] == True: # if Eve has performed a measurement in the k-th circuit
        if ePatterns[0].search(res): # check if the key is '00..'
            eveResults[k][0] = -1 # Alice's qubit measurement result 
            eveResults[k][1] = -1 # Bob's qubit measurement result
        if ePatterns[1].search(res):
            eveResults[k][0] = 1
            eveResults[k][1] = -1
        if ePatterns[2].search(res):
            eveResults[k][0] = -1
            eveResults[k][1] = 1
        if ePatterns[3].search(res):
            eveResults[k][0] = 1
            eveResults[k][1] = 1

In [13]:
## Measurement choises check stage

# Alice and Bob reveal their strings with basis choices
# If in the k-th measurement A and B used the same basis, then they record the result of the k-th measurement as the bit of the sifted key 

aliceSiftedKey = [] # Alice's siffted key string
bobSiftedKey = [] # Bob's siffted key string
eveKeys = [] # Eve's keys; the 1-st column is the Alice's key, the 2-nd is Bob's 

# the sifted key consists of the results obtained by projecting onto the same eigenstates (A2+B1 and A3+B2 measurements)
for k in range(measurementChoicesLength):
    if (aliceMeasurementChoices[k] == 2 and bobMeasurementChoices[k] == 1) or (aliceMeasurementChoices[k] == 3 and bobMeasurementChoices[k] == 2):
        aliceSiftedKey.append(aliceResults[k]) # record Alice's k-th result as a key bit
        bobSiftedKey.append(-bobResults[k]) # record multiplied by -1 Bob's k-th k-th result as a key bit
        eveKeys.append([eveResults[k][0], -eveResults[k][1]]) # record Eve's k-th bits of the key
        
siftedKeyLength = len(aliceSiftedKey) # length of the sifted key

In [14]:
## Comparing bits of the sifted key

# Alice, Bob and Eve do not have this information

abKeyMismatches = 0 # number of mismatching bits in Alice's and Bob's sifted keys
eaKeyMismatches = 0
ebKeyMismatches = 0
eeKeyMismatches = 0

for k in range(siftedKeyLength):
    if aliceSiftedKey[k] != bobSiftedKey[k]:
        abKeyMismatches += 1
    if eveKeys[k][0] != aliceSiftedKey[k]:
        eaKeyMismatches += 1
    if eveKeys[k][1] != bobSiftedKey[k]:
        ebKeyMismatches += 1
    if eveKeys[k][0] != eveKeys[k][1]:
        eeKeyMismatches += 1

In [15]:
## CHSH inequality test

# lists with the counts of measurement results
# each element represents the number of (-1,-1), (-1,1), (1,-1) and (1,1) results respectively
countA1B1 = [0, 0, 0, 0]
countA1B3 = [0, 0, 0, 0]
countA3B1 = [0, 0, 0, 0]
countA3B3 = [0, 0, 0, 0]

# numbers of measurement results obtained from the measurements in a particular basis
count11 = 0
count13 = 0
count31 = 0
count33 = 0

for k in range(measurementChoicesLength):

    res = list(result.get_counts(circuits[k]).keys())[0]
    
    if (aliceMeasurementChoices[k] == 1 and bobMeasurementChoices[k] == 1):
        for j in range(4):
            if abPatterns[j].search(res):
                countA1B1[j] += 1
                count11 += 1
                
    if (aliceMeasurementChoices[k] == 1 and bobMeasurementChoices[k] == 3):
        for j in range(4):
            if abPatterns[j].search(res):
                countA1B3[j] += 1
                count13 += 1        
      
    if (aliceMeasurementChoices[k] == 3 and bobMeasurementChoices[k] == 1):
        for j in range(4):
            if abPatterns[j].search(res):
                countA3B1[j] += 1
                count31 += 1
                
    if (aliceMeasurementChoices[k] == 3 and bobMeasurementChoices[k] == 3):
        for j in range(4):
            if abPatterns[j].search(res):
                countA3B3[j] += 1
                count33 += 1

# expected values of A1B1, A1B3, A3B1 and A3B3  measurements results
expect11 = (countA1B1[0] - countA1B1[1] - countA1B1[2] + countA1B1[3])/count11 # -1/sqrt(2)
expect13 = (countA1B3[0] - countA1B3[1] - countA1B3[2] + countA1B3[3])/count13 # 1/sqrt(2)
expect31 = (countA3B1[0] - countA3B1[1] - countA3B1[2] + countA3B1[3])/count31 # -1/sqrt(2)
expect33 = (countA3B3[0] - countA3B3[1] - countA3B3[2] + countA3B3[3])/count33 # -1/sqrt(2)

# CHSH correlation; must be equal to 2*sqrt(2) in the ideal case
corr = expect11 - expect13 + expect31 + expect33
diff = abs(abs(corr) - 2*math.sqrt(2))

In [16]:
## Print results

# CHSH inequality test
print('CHSH correlation value: ' + str(round(corr, 3)))
print('Difference from -2*sqrt(2): ' + str(round(diff, 3)) + '\n')

# Sifted keys
print('Length of the sifted key: ' + str(siftedKeyLength))
print('Number of mismatching bits: ' + str(abKeyMismatches) + '\n')

print('QBER: ' + str(round(abKeyMismatches/siftedKeyLength * 100, 2)) + ' %')
print('Eve\'s knowledge: ' + str(round((eaKeyMismatches+ebKeyMismatches)/2/siftedKeyLength * 100, 2)) + ' %\n')

# Eve's keys characteristics
#print('eaKeyMismatches: ' + str(eaKeyMismatches))
#print('ebKeyMismatches: ' + str(ebKeyMismatches))
#print('eeKeyMismatches: ' + str(eeKeyMismatches))

CHSH correlation value: -1.454
Difference from -2*sqrt(2): 1.374

Length of the sifted key: 111
Number of mismatching bits: 15

QBER: 13.51 %
Eve's knowledge: 96.4 %



In [17]:
## Check measurement errors (actual for execution on a real device)

# The measurement in the A2B1 and A3B2 basises can give only the (-1,1) and (1,-1) results.
# This block counts how many (-1,-1) and (1,1) results obtained after the A2B1 and A3B2 measurements.

mismatchesList = []

for k in range(measurementChoicesLength):

    res = list(result.get_counts(circuits[k]).keys())[0]
    
    result01 = False # by default
    result10 = False
    
    if abPatterns[1].search(res):
        result01 = True
    if abPatterns[2].search(res):
        result10 = True
    
    condition1 = ('A2B1' in circuits[k] or 'A3B2' in circuits[k]) and not result01 # if result is not (-1,1)
    condition2 = ('A2B1' in circuits[k] or 'A3B2' in circuits[k]) and not result10 # if result is not (1,-1)
    
    if condition1 and condition2 == True:
        mismatchesList.append([str(circuits[k]), res])
        
pd.DataFrame(mismatchesList,columns = ['Measurement', 'Result'])

,Measurement,Result
